In [ ]:
#r "./../../../../../../public/src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
using System;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Solution;
using BoSSS.Application.XNSE_Solver;
using BoSSS.Application.BoSSSpad;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Control;
using BoSSS.Solution.XNSECommon;
using BoSSS.Solution.NSECommon;
using BoSSS.Application.XNSE_Solver.LoadBalancing;
using BoSSS.Solution.LevelSetTools;
using BoSSS.Solution.XdgTimestepping;

using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

In [ ]:
struct Parameterz{
    public Parameterz(int _Cores, int _Poly, int _Res){
        Cores = _Cores;
        Poly = _Poly;
        Res = _Res;
    }
    public int Cores;
    public int Poly;
    public int Res;
}

In [ ]:
// set parameterz
var Parameterz = new List<Parameterz>();

/*
Values for stretch in x-direction = 1, meaning: cells = Res * stretch * Res * Res

k2	cores	N	Res		cells	DOF per core	
	4	34	10.55667192	11	1331	11314	
	8	34	13.30057317	14	2744	11662	
	16	34	16.75767211	17	4913	10440	
	32	34	21.11334384	22	10648	11314	
	64	34	26.60114634	28	21952	11662	
	128	34	33.51534422	34	39304	10440	
	256	34	42.22668768	44	85184	11314	
	512	34	53.20229267	56	175616	11662	
	1024	34	67.03068844	68	314432	10440	
k3	cores	N	Res		cells	DOF per core	
	4	70	8.298265334	9	729	12758	
	8	70	10.45515917	11	1331	11646	
	16	70	13.17267512	14	2744	12005	
	32	70	16.59653067	18	5832	12758	
	64	70	20.91031834	22	10648	11646	
	128	70	26.34535024	28	21952	12005	
	256	70	33.19306133	36	46656	12758	
	512	70	41.82063669	44	85184	11646	
	1024	70	52.69070048	56	175616	12005	
k4	cores	N	Res		cells	DOF per core	
	4	125	6.839903787	7	343	10719	
	8	125	8.61773876	9	729	11391	
	16	125	10.85767047	11	1331	10398	
	32	125	13.67980757	14	2744	10719	
	64	125	17.23547752	18	5832	11391	
	128	125	21.71534093	22	10648	10398	
	256	125	27.35961515	28	21952	10719	
	512	125	34.47095504	36	46656	11391
*/

 // 8 cores
 //Parameterz.Add(new Parameterz(8,2,14));
 //Parameterz.Add(new Parameterz(8,3,11));
 Parameterz.Add(new Parameterz(8,4,9));

 // 16 cores
 //Parameterz.Add(new Parameterz(16,2,18));
 //Parameterz.Add(new Parameterz(16,3,14));
 Parameterz.Add(new Parameterz(16,4,11));
 
 // 32 cores
 //Parameterz.Add(new Parameterz(32,2,22));
 //Parameterz.Add(new Parameterz(32,3,18));
 Parameterz.Add(new Parameterz(32,4,14));

// 64 cores
 //Parameterz.Add(new Parameterz(64,2,28));
 //Parameterz.Add(new Parameterz(64,3,22));
 Parameterz.Add(new Parameterz(64,4,18));

 // 128 cores
 //Parameterz.Add(new Parameterz(128,2,36));
 //Parameterz.Add(new Parameterz(128,3,28));
 Parameterz.Add(new Parameterz(128,4,22));

 // 256 cores
 //Parameterz.Add(new Parameterz(256,2,44));
 //Parameterz.Add(new Parameterz(256,3,36));
 Parameterz.Add(new Parameterz(256,4,28));

/*
 // 512 cores
 Parameterz.Add(new Parameterz(512,2,56));
 Parameterz.Add(new Parameterz(512,3,44));
 Parameterz.Add(new Parameterz(512,4,36));
 */

In [ ]:
bool useAMR = false;
bool useLoadBal = true;
int NoOfTimeSteps = 1;
bool Steady = true;
bool IncludeConvection = false;
var Gshape = Shape.Sphere;

In [ ]:
ExecutionQueues

index,type,Username,Password,ServerName,PrivateKeyFilePath,AdditionalBatchCommands,DeploymentBaseDirectoryAtRemote,SlurmAccount,Email,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,MonoDebug,AllowedDatabasesPaths,BatchInstructionDir,ComputeNodes,DefaultJobPriority,SingleNode
0,BoSSS.Application.BoSSSpad.SlurmClient,jw52xeqa,<null>,lcluster14.hrz.tu-darmstadt.de,C:\Users\weber\.ssh\id_rsa,"[ #SBATCH -p test24, #SBATCH -C avx512, #SBATCH --mem-per-cpu=2000 ]",/home/jw52xeqa/Deployerie,special00006,<null>,X:\Deployerie,True,HHLR,dotnet,False,"[ { W:\work\scratch\jw52xeqa == /work/scratch/jw52xeqa: LocalMountPath: W:\work\scratch\jw52xeqa, PathAtRemote: /work/scratch/jw52xeqa } ]",,,,
1,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,,,,,,,,,C:\Users\weber\AppData\Local\BoSSS-LocalJobs,False,LocalPC,dotnet,,,<null>,,,
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,FDY\weber,,DC2,,,,,,\\hpccluster\hpccluster-scratch\weber\dply,True,FDY-cluster,dotnet,,"[ { \\hpccluster\hpccluster-scratch\weber\trash_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\weber\trash_DB, PathAtRemote: } ]",,[ HPCCLUSTER2 ],Normal,True
3,BoSSS.Application.BoSSSpad.SlurmClient,jw52xeqa,<null>,lcluster5.hrz.tu-darmstadt.de,C:\Users\weber\.ssh\id_rsa,<null>,/home/jw52xeqa/Deployerie,project01217,<null>,X:\Deployerie,True,<null>,dotnet,False,,,,,


In [ ]:
// ==================================
// setup Client & Workflow & Database
// ==================================
/*
var myBatch = (SlurmClient)ExecutionQueues[3];
var AddSbatchCmds = new List<string>();
AddSbatchCmds.AddRange(new string[]{"#SBATCH -p test24", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu="+MemoryPerCore});
myBatch.AdditionalBatchCommands = AddSbatchCmds.ToArray();
myBatch.AdditionalBatchCommands
*/

In [ ]:
// Config for Jenkins-Server
//var myBatch = (MsHPC2012Client)ExecutionQueues[2];
var myBatch = GetDefaultQueue();
if(myBatch is SlurmClient){
    (myBatch as SlurmClient).AdditionalBatchCommands = new string[]{"#SBATCH -p test24", "#SBATCH -C avx512", "#SBATCH --mem-per-cpu=2000"};
}

In [ ]:
string WFlowName = "weak_scaling_5_1_p-MG";
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.Init(WFlowName);
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.SetNameBasedSessionJobControlCorrelation();

In [ ]:
//string dirname = "DB_rotCubePaper";
//string dirname = "DB_rotSphere_CoreScaling";
//string dirname = "DB_rotSphere_DOFScaling";
//string dirname ="DB_rotSphereBenchmark";
//string dirname ="DB_rotSphere_CoreScaling";

In [ ]:
//myBatch.AllowedDatabasesPaths.Clear();
//string winpath = @"W:\work\scratch\jw52xeqa\"+dirname;
//string winpath = @"\\hpccluster\hpccluster-scratch\"+dirname;
//string remotepath = @"/work/scratch/jw52xeqa/"+dirname;
//var myDB = OpenOrCreateDatabase(winpath);
//myBatch.AllowedDatabasesPaths.Add(new AllowedDatabasesPair(winpath,remotepath)); myBatch.AllowedDatabasesPaths

In [ ]:
var pair = myBatch.AllowedDatabasesPaths.Pick(0);
string DBName = @"\"+"DB_rotSphere_CoreScaling";
string localpath=pair.LocalMountPath+DBName;
string remotepath=pair.PathAtRemote+DBName;
var myDB = OpenOrCreateDatabase(localpath); myDB.Path

W:\work\scratch\jw52xeqa\DB_rotSphere_CoreScaling

In [ ]:
static class Utils {
    // DOF per cell in 3D
    public static int Np(int p) {
        return (p*p*p + 6*p*p + 11*p + 6)/6;
    }    
}

using 

In [ ]:
double xMax = 4.0, yMax = 1.0, zMax = 1.0;
double xMin = -2.0, yMin = -1.0,zMin = -1.0;

double xMax = 1.0, yMax = 1.0, zMax=1.0;
double xMin = -1.0, yMin = -1.0,zMin = -1.0;

In [ ]:
static Dictionary<int, IGridInfo> Grids = new Dictionary<int, IGridInfo>();
foreach(var P in Parameterz){
    int Res = P.Res;
    if(Grids.TryGetValue(Res,out IGridInfo ignore))
        continue;
    //int Stretching = (int)Math.Floor(Math.Abs(xMax-xMin)/Math.Abs(yMax-yMin));
    int Stretching = 1;
    var _xNodes = GenericBlas.Linspace(xMin, xMax, Stretching*Res + 1);
    var _yNodes = GenericBlas.Linspace(yMin, yMax, Res + 1);
    var _zNodes = GenericBlas.Linspace(zMin, zMax, Res + 1);

    GridCommons grd;
    string gname = "RotBenchmarkGrid";
    
    var tmp = new List<IGridInfo>();
    foreach(var grid in myDB.Grids){
        try{
            bool IsMatch = grid.Name.Equals(gname)&&grid.NumberOfCells==(_xNodes.Length-1)*(_yNodes.Length-1)*(_zNodes.Length-1);
            if(IsMatch) tmp.Add(grid);
        }
        catch(Exception ex) {
            Console.WriteLine(ex.Message);
        }
    }
    //var tmp = myDB.Grids.Where(g=>g.Name.Equals(gname)&&g.NumberOfCells==Res*Res*Res); // this leads to exception in case of broken grids
    if(tmp.Count()>=1){
        Console.WriteLine("Grid found: "+tmp.Pick(0).Name);
        Grids.Add(Res,tmp.Pick(0));
        continue;
    }
    
    grd = Grid3D.Cartesian3DGrid(_xNodes, _yNodes, _zNodes);
    grd.Name = gname;
    //grd.AddPredefinedPartitioning("debug", MakeDebugPart);

    grd.EdgeTagNames.Add(1, "Velocity_inlet");
    grd.EdgeTagNames.Add(2, "Wall");
    grd.EdgeTagNames.Add(3, "Pressure_Outlet");

    grd.DefineEdgeTags(delegate (double[] _X) {
        var X = _X;
        double x, y, z;
        x = X[0];
        y = X[1];
        z = X[2];
        if(Math.Abs(x-xMin)<1E-8)
            return 1;
        else
            return 3;
    });
    myDB.SaveGrid(ref grd,false);
    Grids.Add(Res,grd);
} Grids.Keys.ToList()

index,value
0,9
1,11
2,14
3,18
4,22
5,28


In [ ]:
int SpaceDim = 3; 

In [ ]:
Func<IGridInfo, int, XNSE_Control> GenXNSECtrl = delegate(IGridInfo grd, int k){
    XNSE_Control C = new XNSE_Control();
    // basic database options
    // ======================
    C.AlternateDbPaths = new[] {
        (localpath, ""),
        (remotepath,"")};
    //C.AlternateDbPaths = new[] {
    //    (@"/work/scratch/jw52xeqa/DB_rotSphereBenchmark", ""),
    //    (@"W:\work\scratch\jw52xeqa\DB_rotSphereBenchmark","")};
    //C.DbPath=@"\\hpccluster\hpccluster-scratch\DB_rotSphereBenchmark";
    C.savetodb = true;
    int J  = grd.NumberOfCells;
    C.SessionName = string.Format("J{0}_k{1}_t{2}", J, k,NoOfTimeSteps);
    if(IncludeConvection){
        C.SessionName += "_NSE";
        C.Tags.Add("NSE");
    } else {
        C.SessionName += "_Stokes";
        C.Tags.Add("Stokes");
    }
    C.Tags.Add(SpaceDim + "D");
    if(Steady)C.Tags.Add("steady");
    else C.Tags.Add("transient");
    C.Tags.Add("reortho_Iter2_sameRes");

    // DG degrees
    // ==========
    C.SetFieldOptions(k, Math.Max(k, 2));
    C.saveperiod = 1;
    //C.TracingNamespaces = "*";

    C.GridGuid = grd.ID;
    C.GridPartType = GridPartType.clusterHilbert;
    C.DynamicLoadbalancing_ClassifierType = ClassifierType.CutCells;
    C.DynamicLoadBalancing_On = useLoadBal;
    C.DynamicLoadBalancing_RedistributeAtStartup = true;
    C.DynamicLoadBalancing_Period = 1;
    C.DynamicLoadBalancing_ImbalanceThreshold = 0.1;

    // Physical Parameters
    // ===================
    const double rhoA = 1;
    const double Re = 50;
    double muA = 1;
    
    double partRad = 0.3001;
    double anglev = Re*muA/rhoA/(2*partRad);
    //double anglev = 0.0;
    double d_hyd = 2*partRad;
    double VelocityIn = Re*muA/rhoA/d_hyd;
    double[] pos = new double[SpaceDim];

    C.PhysicalParameters.IncludeConvection = IncludeConvection;
    C.PhysicalParameters.Material = true;
    C.PhysicalParameters.rho_A = rhoA;
    C.PhysicalParameters.mu_A = muA;

    C.Rigidbody.SetParameters(pos,anglev,partRad,SpaceDim);
    C.Rigidbody.SpecifyShape(Gshape);
    C.Rigidbody.SetRotationAxis("x");

    C.AddInitialValue(VariableNames.LevelSetCGidx(0), new Formula("X => -1"));
    C.UseImmersedBoundary = true;
    
    C.AddInitialValue("Pressure", new Formula(@"X => 0"));
    C.AddBoundaryValue("Pressure_Outlet");
    C.AddBoundaryValue("Velocity_inlet","VelocityX",new Formula($"(X) => {VelocityIn}"));

    C.CutCellQuadratureType = BoSSS.Foundation.XDG.XQuadFactoryHelper.MomentFittingVariants.Saye;
    C.UseSchurBlockPrec = true;
    C.AgglomerationThreshold = 0.1;
    C.AdvancedDiscretizationOptions.ViscosityMode = ViscosityMode.FullySymmetric;
    C.Option_LevelSetEvolution2 = LevelSetEvolution.Prescribed;
    C.Option_LevelSetEvolution = LevelSetEvolution.None;
    C.Timestepper_LevelSetHandling = LevelSetHandling.None;
    C.LinearSolver.NoOfMultigridLevels = 4;
    C.LinearSolver.ConvergenceCriterion = 1E-6;
    C.LinearSolver.MaxSolverIterations = 500;
    C.LinearSolver.MaxKrylovDim = 50;
    C.LinearSolver.TargetBlockSize = 1000;
    C.LinearSolver.verbose = true;
    C.LinearSolver.SolverCode = LinearSolverCode.exp_Kcycle_schwarz;
    C.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
    C.NonLinearSolver.ConvergenceCriterion = 1E-6;
    C.NonLinearSolver.MaxSolverIterations = 10;
    C.NonLinearSolver.verbose = true;

    C.AdaptiveMeshRefinement = useAMR;
    if (useAMR) {
        C.SetMaximalRefinementLevel(1);
        C.AMR_startUpSweeps = 0;
    }

    // Timestepping
    // ============
    double dt = -1;
    if(Steady){
        C.TimesteppingMode = AppControl._TimesteppingMode.Steady;
        dt = 1000;
        C.NoOfTimesteps = 1;
    } else {
        C.TimesteppingMode = AppControl._TimesteppingMode.Transient;        
        dt = 0.1;        
        C.NoOfTimesteps = NoOfTimeSteps;
    }
    C.TimeSteppingScheme = TimeSteppingScheme.ImplicitEuler;
    C.dtFixed = dt;
    return C;
};

In [ ]:
var controls = new Dictionary<Parameterz,XNSE_Control>();
foreach(var P in Parameterz){
    int k = P.Poly;
    Grids.TryGetValue(P.Res,out IGridInfo grd);
    controls.Add(P,GenXNSECtrl(grd,k));
} controls.Values.Select(s=>s.SessionName)

index,value
0,J729_k4_t1_Stokes
1,J1331_k4_t1_Stokes
2,J2744_k4_t1_Stokes
3,J5832_k4_t1_Stokes
4,J10648_k4_t1_Stokes
5,J21952_k4_t1_Stokes


In [ ]:
static Func<KeyValuePair<Parameterz,XNSE_Control>,int> NodeRegression =  delegate (KeyValuePair<Parameterz,XNSE_Control> extcontrol) {

    var thisctrl = extcontrol;
    Grids.TryGetValue(thisctrl.Key.Res,out IGridInfo grd);
    int Ncells = grd.NumberOfCells;
    int k = thisctrl.Key.Poly;
    int DOFperCell = Utils.Np(k);
    int DOFperCore = DOFperCell*Ncells;
    double mempercore = -1E-10*DOFperCore*DOFperCore+2.6E-3*DOFperCore+13500;
    double memoryneed = mempercore * thisctrl.Key.Cores;
    double memorypernode = 384*1024; // MB
    int tmp = (int)Math.Ceiling(memoryneed / memorypernode); 
    double bla = Math.Ceiling(Math.Log(tmp)/Math.Log(2));
    int nodes2allocate = (int)Math.Pow(2,bla);
    
    return nodes2allocate;
};

In [ ]:
NodeRegression(controls.Pick(0))

1

In [ ]:
controls.Select(s=>s.Value.SessionName)

index,value
0,J729_k4_t1_Stokes
1,J1331_k4_t1_Stokes
2,J2744_k4_t1_Stokes
3,J5832_k4_t1_Stokes
4,J10648_k4_t1_Stokes
5,J21952_k4_t1_Stokes


In [ ]:
int iSweep=0;
foreach(var ctrl in controls){
    try{
    int cores= ctrl.Key.Cores;
    var ctrlobj = ctrl.Value;
    string sessname = ctrlobj.SessionName;
    ctrlobj.SessionName = sessname + "_c"+cores+"_Re50_"+"mue_"+ctrlobj.PhysicalParameters.mu_A;
    var aJob   = new Job("rotSphereInlet_"+Gshape+ctrlobj.SessionName,typeof(XNSE));
    aJob.SetControlObject(ctrlobj);
    aJob.NumberOfMPIProcs         = cores;
    aJob.ExecutionTime            = "3:00:00";
    aJob.UseComputeNodesExclusive = true;

    if(myBatch is SlurmClient){
        int nodes2allocate = NodeRegression.Invoke(ctrl);

        var Cmdtmp = new List<string>();
        Cmdtmp.AddRange((myBatch as SlurmClient).AdditionalBatchCommands.ToList());
        Cmdtmp.Add($"#SBATCH -N {nodes2allocate}");
        (myBatch as SlurmClient).AdditionalBatchCommands = Cmdtmp.ToArray();
    }
    aJob.Activate(myBatch);
    iSweep++;
    } catch (Exception ex){
        Console.WriteLine(ex.Message);
    }
}

In [ ]:
BoSSS.Application.BoSSSpad.BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate();